### Análisis de tiempos de autosurtido Noviembre 2020 - Enero 2021
#### Este análisis tiene como objetivo encontrar los talleres/agencias que cumplen con los tiempos meta de autosurtido y cuántos cumplen con las fechas promesa de entrega de los vehículos.

In [127]:
import pandas as pd
# Glancing at the excel file that contains information about 74 issues
excel = pd.read_excel(r"D:\Users\430008252\OneDrive - HDI Seguros\Documentos\AutoSurtido\CDR-AUTOSURTIDO_18-01-2021.xlsx")
excel.head()

,NIP_SUCURSAL,RFC,RAZONSOCIAL,NOMBRECOMERCIAL_CDR,OFICINAASEGURADORA,REGIONAL,MONTOAUTOSURTIDO
0,298742-25478,SLP860522482,SANROMAN LAMINADOS Y PINTURAS ESPECIALIZADOS S...,SANROMAN LAMINADOS Y PINTURAS ESPECIALIZADOS,LEON,BAJIO,13000
1,599516-22329,SASM800712BW3,MIGUEL SANROMAN SASHIDA,SANROMAN LAS TORRES,LEON,BAJIO,13000
2,1947654-36682,LAM100601LF5,LAMINAUTOS SA DE CV,LAMINAUTOS,LEON,BAJIO,13000
3,2831677-37961,VIPA801021FJ5,ALBERTO VILLASENOR PANDURO,QUALITY CAR MORELIA,MORELIA,OCCIDENTE,10500
4,2785226-37885,FAU931014QC1,FUERZA AUTOMOTRIZ SA DE CV,FUERZA AUTOMOTRIZ MICHOACAN,MORELIA,OCCIDENTE,10500


In [128]:
# Importing libraries to query from DWH
import pyodbc
# Settings to connect with the database
server = 'data.hdi.com.mx'
database = 'Procesos'
DWH = pyodbc.connect(r'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';TRUSTED_CONNECTION=TRUE')
# Queries from DWH
Tb_BI_GrlSinValuacion = pd.read_sql_query("""SELECT
    ValLst_NumeroExpediente
    , IdOficinaAtencion
    , FechaAutorizacionValuacion,FechaSolicitudRefacc
    , FechaEntregaRefacc
    , FechaPromesa
    , FechaEntrega
    , MontoRefacciones
    , MontoManoObra
    , NumeroTotalPiezas
    , TipoCentroReparacion
    , CONCAT(NipTaller, SucursalTaller) AS NipSucursal
    , Taller
    
    FROM Tb_BI_GrlSinValuacion
    
    WHERE
        FechaAutorizacionValuacion >= '2020-11-01 00:00:00'
        AND FechaAutorizacionValuacion < '2021-02-01'
        AND TipoProceso = 'EN REPARACION'
        AND IdLineaNegocio = 4""",DWH).dropna()
Vw_Bi_GrlSinValuacion_Refacciones = pd.read_sql_query("""SELECT
    [ID Valuación]
    , [Oficina de Atención]
    , [Nombre taller]
    , [Descripción Homologada (calculado)]
    , [Origen de la refacción (Genérica, Original, TOT, Usada)]
    , FechaPromesaEntrega
    , FechaRecepcion
    , FechaAsignacion
    , FechaEntregaProveedor
    , CONCAT(NipProveedor, IdSucursal) AS NipSucursal

    FROM Vw_Bi_GrlSinValuacion_Refacciones

    WHERE 
        [ID Valuación] IN 
            (SELECT ValLst_NumeroExpediente
            
            FROM Tb_BI_GrlSinValuacion
            
            WHERE FechaAutorizacionValuacion >= '2020-11-01 00:00:00'
            AND FechaAutorizacionValuacion < '2021-02-01'
            AND TipoProceso = 'EN REPARACION'
            AND IdLineaNegocio = 4)""",DWH).dropna()
Vw_bi_SinModeloMetas = pd.read_sql_query("""SELECT

    Periodo
    , Oficina
    , TiempoSurtidoRefacciones
    , CumplimientoFechaPromesa

    FROM Vw_bi_SinModeloMetas

    WHERE Periodo >= 202011
    AND Periodo < 202102""",DWH).dropna()

#### La tabla Tb_BI_GrlSinValuacion contiene información individual de cada valuación

In [129]:
# Glancing at the data frame
Tb_BI_GrlSinValuacion.head()

,ValLst_NumeroExpediente,IdOficinaAtencion,FechaAutorizacionValuacion,FechaSolicitudRefacc,FechaEntregaRefacc,FechaPromesa,FechaEntrega,MontoRefacciones,MontoManoObra,NumeroTotalPiezas,TipoCentroReparacion,NipSucursal,Taller
0,1-21A1,6.0,2021-01-09 12:48:23.440,2021-01-14 14:09:15.313,2021-01-27 11:40:56.670,2021-02-17,2021-02-09 09:15:46.887,12093.84,3821.19,10.0,AGENCIA,92689836286,AUTOS GP IRAPUATO
2,118157-20A1,5.0,2020-12-05 10:50:34.537,2020-12-05 10:50:31.857,2021-01-06 12:50:51.640,2020-12-29,2021-01-09 12:08:57.500,13827.00,4172.28,7.0,TALLER,128435233986,BOUTIQUE CAR SHOW
4,1770-21A1,7.0,2021-01-21 22:48:59.903,2021-01-21 22:48:59.633,2021-02-02 17:27:38.217,2021-02-08,2021-02-09 09:41:33.603,8505.77,4083.85,5.0,AGENCIA,125418834504,VW LOMAS VERDES
5,188623-20A1,22.0,2020-11-03 20:11:49.277,2020-11-03 20:11:49.010,2020-11-11 11:13:04.120,2020-11-24,2020-11-14 10:53:10.780,8585.00,4728.28,9.0,TALLER,92894732414,CROMO TEAM
6,188630-20A1,3.0,2020-11-03 08:42:38.333,2020-11-06 11:02:46.257,2020-11-03 12:38:04.810,2020-11-13,2020-11-12 10:58:40.480,20470.90,7239.96,15.0,AGENCIA,28425932022,HONDA ROCA


In [130]:
# Checking the data types in each column
Tb_BI_GrlSinValuacion.IdOficinaAtencion = Tb_BI_GrlSinValuacion.IdOficinaAtencion.astype("int")
Tb_BI_GrlSinValuacion.dtypes

ValLst_NumeroExpediente               object
IdOficinaAtencion                      int32
FechaAutorizacionValuacion    datetime64[ns]
FechaSolicitudRefacc          datetime64[ns]
FechaEntregaRefacc            datetime64[ns]
FechaPromesa                  datetime64[ns]
FechaEntrega                  datetime64[ns]
MontoRefacciones                     float64
MontoManoObra                        float64
NumeroTotalPiezas                    float64
TipoCentroReparacion                  object
NipSucursal                           object
Taller                                object
dtype: object

In [131]:
# Setting data type values to the correct ones
Tb_BI_GrlSinValuacion = Tb_BI_GrlSinValuacion.astype({"ValLst_NumeroExpediente":"category","IdOficinaAtencion":"category","NumeroTotalPiezas":"int","TipoCentroReparacion":"category", "NipSucursal":"category","Taller":"category"})
Tb_BI_GrlSinValuacion.dtypes

ValLst_NumeroExpediente             category
IdOficinaAtencion                   category
FechaAutorizacionValuacion    datetime64[ns]
FechaSolicitudRefacc          datetime64[ns]
FechaEntregaRefacc            datetime64[ns]
FechaPromesa                  datetime64[ns]
FechaEntrega                  datetime64[ns]
MontoRefacciones                     float64
MontoManoObra                        float64
NumeroTotalPiezas                      int32
TipoCentroReparacion                category
NipSucursal                         category
Taller                              category
dtype: object

#### La tabla Vw_Bi_GrlSinValuacion_Refacciones contiene información de todas las refacciones asociadas a una valuación. 

In [132]:
# Glancing at the data frame
Vw_Bi_GrlSinValuacion_Refacciones.head()

,ID Valuación,Oficina de Atención,Nombre taller,Descripción Homologada (calculado),"Origen de la refacción (Genérica, Original, TOT, Usada)",FechaPromesaEntrega,FechaRecepcion,FechaAsignacion,FechaEntregaProveedor,NipSucursal
0,196211-20A1,8.0,"UNITED AUTO DE MONTERREY, S DE RL DE CV",TOLVA,GENUINA,2000-01-02,2020-11-18 18:07:33.247,2020-11-18 16:12:20.917,2020-11-18 18:08:22.837,31707934476
1,196211-20A1,8.0,"UNITED AUTO DE MONTERREY, S DE RL DE CV",MOTOR,GENUINA,2000-01-02,2020-11-18 18:07:30.740,2020-11-18 16:12:20.917,2020-11-18 18:08:23.257,31707934476
2,196211-20A1,8.0,"UNITED AUTO DE MONTERREY, S DE RL DE CV",TOLVA,GENUINA,2000-01-02,2020-11-18 18:07:33.083,2020-11-18 16:12:20.917,2020-11-18 18:08:22.367,31707934476
3,196211-20A1,8.0,"UNITED AUTO DE MONTERREY, S DE RL DE CV",GUARDAFANGO,GENUINA,2000-01-02,2020-11-18 18:07:27.380,2020-11-18 16:12:20.917,2020-11-18 18:08:21.843,31707934476
4,196211-20A1,8.0,"UNITED AUTO DE MONTERREY, S DE RL DE CV",RADIADOR,GENUINA,2000-01-02,2020-11-18 18:07:21.703,2020-11-18 16:12:20.917,2020-11-18 18:08:21.043,31707934476


In [133]:
# Checking the data types in each columns
Vw_Bi_GrlSinValuacion_Refacciones["Oficina de Atención"] = Vw_Bi_GrlSinValuacion_Refacciones["Oficina de Atención"].astype("int")
Vw_Bi_GrlSinValuacion_Refacciones.dtypes

ID Valuación                                                       object
Oficina de Atención                                                 int32
Nombre taller                                                      object
Descripción Homologada (calculado)                                 object
Origen de la refacción (Genérica, Original, TOT, Usada)            object
FechaPromesaEntrega                                        datetime64[ns]
FechaRecepcion                                             datetime64[ns]
FechaAsignacion                                            datetime64[ns]
FechaEntregaProveedor                                      datetime64[ns]
NipSucursal                                                        object
dtype: object

In [134]:
# Setting data type values to the correct ones
Vw_Bi_GrlSinValuacion_Refacciones = Vw_Bi_GrlSinValuacion_Refacciones.astype({"ID Valuación":"category", "Oficina de Atención":"category","Nombre taller":"category","Descripción Homologada (calculado)":"category","Origen de la refacción (Genérica, Original, TOT, Usada)":"category", "NipSucursal":"category"})
Vw_Bi_GrlSinValuacion_Refacciones.dtypes

ID Valuación                                                     category
Oficina de Atención                                              category
Nombre taller                                                    category
Descripción Homologada (calculado)                               category
Origen de la refacción (Genérica, Original, TOT, Usada)          category
FechaPromesaEntrega                                        datetime64[ns]
FechaRecepcion                                             datetime64[ns]
FechaAsignacion                                            datetime64[ns]
FechaEntregaProveedor                                      datetime64[ns]
NipSucursal                                                      category
dtype: object

#### La tabla Vw_bi_SinModeloMetas contiene información de los tiempos meta de surtido por oficina

In [135]:
# Glancing at the data frame
Vw_bi_SinModeloMetas.head()

,Periodo,Oficina,TiempoSurtidoRefacciones,CumplimientoFechaPromesa
0,202011,3,5.60,0.72
1,202011,4,5.12,0.72
2,202011,5,5.00,0.72
3,202011,6,6.16,0.72
4,202011,7,5.76,0.72


### Para este análisis se buscará cuántas refacciones cumplen con los tiempos meta de surtido.

De la tabla Vw_Bi_GrlSinValuacion_Refacciones, se evaluará -donde sea posible- qué tantos proveedores sí cumplen con sus fechas de entrega (en los casos que sea posible).

De la tabla Tb_BI_GralSinValuacion se evaluará cuántos talleres cumplieron con los tiempos meta utilizando también la tabla Vw_bi_SinModelosMetas.

#### El primer paso será asignar como NULL todas las fechas de Vw_Bi_GralSinValuacion correspondientes al año 2000

In [145]:
# Assigning pandas.Not a Time value to 2000 year dates
Vw_Bi_GrlSinValuacion_Refacciones.loc[Vw_Bi_GrlSinValuacion_Refacciones['FechaAsignacion']<min(Tb_BI_GrlSinValuacion.FechaAutorizacionValuacion),'FechaAsignacion']= pd.NaT
Vw_Bi_GrlSinValuacion_Refacciones.loc[Vw_Bi_GrlSinValuacion_Refacciones['FechaPromesaEntrega']<min(Tb_BI_GrlSinValuacion.FechaAutorizacionValuacion),'FechaPromesaEntrega']= pd.NaT
Vw_Bi_GrlSinValuacion_Refacciones.loc[Vw_Bi_GrlSinValuacion_Refacciones['FechaRecepcion']<min(Tb_BI_GrlSinValuacion.FechaAutorizacionValuacion),'FechaRecepcion']= pd.NaT
Vw_Bi_GrlSinValuacion_Refacciones.loc[Vw_Bi_GrlSinValuacion_Refacciones['FechaEntregaProveedor']<min(Tb_BI_GrlSinValuacion.FechaAutorizacionValuacion),'FechaEntregaProveedor']= pd.NaT

In [162]:
# Complete data set (all Dates are valid)
refacCompleto = Vw_Bi_GrlSinValuacion_Refacciones[pd.notnull(Vw_Bi_GrlSinValuacion_Refacciones.FechaAsignacion) & pd.notnull(Vw_Bi_GrlSinValuacion_Refacciones.FechaEntregaProveedor) & pd.notnull(Vw_Bi_GrlSinValuacion_Refacciones.FechaPromesaEntrega)]
# Absolutelly crap data set (none of the date columns is valid)
refacNAS = Vw_Bi_GrlSinValuacion_Refacciones[pd.isnull(Vw_Bi_GrlSinValuacion_Refacciones.FechaAsignacion) & pd.isnull(Vw_Bi_GrlSinValuacion_Refacciones.FechaEntregaProveedor) & pd.isnull(Vw_Bi_GrlSinValuacion_Refacciones.FechaPromesaEntrega)]
# 

In [181]:
# Checking the number of complete cases the main data frame has
print("Del total de datos, el",round(100-(Vw_Bi_GrlSinValuacion_Refacciones.shape[0]-refacCompleto.shape[0])/Vw_Bi_GrlSinValuacion_Refacciones.shape[0] *100,2),"% está completo")
print("El",round(100-(Vw_Bi_GrlSinValuacion_Refacciones.shape[0]-refacNAS.shape[0])/Vw_Bi_GrlSinValuacion_Refacciones.shape[0] *100,2),"% no contiene ninguna fecha para evaluar")


Del total de datos, el 29.11 % está completo
El 0.77 % no contiene ninguna fecha para evaluar


In [175]:
refacNAS.shape[0]
# Vw_Bi_GrlSinValuacion_Refacciones.shape

1072

In [ ]:
# RefaccionesCompleto = Vw_Bi_GrlSinValuacion_Refacciones[]
pd.isnull(Vw_Bi_GrlSinValuacion_Refacciones.FechaPromesaEntrega).any()

In [178]:
round(100-(Vw_Bi_GrlSinValuacion_Refacciones.shape[0]-refacNAS.shape[0])/Vw_Bi_GrlSinValuacion_Refacciones.shape[0] *100,2)

0.77